In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pylab as plt

from modules.utils import load_cifar10, load_cifar100
from modules.cnn_with_spectral_pooling import CNN_Spectral_Pool

% matplotlib inline
% load_ext autoreload
% autoreload 2

# Download CIFAR data, if necessary, and load it

In [2]:
xtrain, ytrain, xtest, ytest = load_cifar100(get_test_data=True, channels_last=False)

file already downloaded..


In [3]:
xtrain.shape, ytrain.shape, xtest.shape, ytest.shape

((50000, 3, 32, 32), (50000,), (10000, 3, 32, 32), (10000,))

# Build graph with optimal hyperparameters

In [4]:
from modules.cnn_with_spectral_pooling import CNN_Spectral_Pool
validation_size = 4096

cnn = CNN_Spectral_Pool(M=7,
                        num_output=100,
                        verbose=True,
                        learning_rate=1.09e-3,
                        l2_norm=2.7e-5,
                        lr_reduction_factor=0.5,
                        lr_reduction_epochs=[11,21,31,41],
                        gamma=0.6336)


In [ ]:
tf.reset_default_graph()
cnn.train(xtrain[:-validation_size],
          ytrain[:-validation_size],
          xtrain[-validation_size:],
          ytrain[-validation_size:],
          batch_size=256,
          epochs=50,
          extra_conv_layer=True,
          use_global_averaging=True,
          model_name='optimal_hyperparams'
)
full_model_name = cnn.full_model_name

Building tf graph...
Adding conv layer 1 | Input size: 32 | Input channels: 3 | #filters: 128 | filter size: 3
Adding spectral pool layer 1 | Input size: 32 | filter size: (20,20) | Freq Dropout Bounds: (3.0642857142857145,11.0)
Adding conv layer 2 | Input size: 20 | Input channels: 128 | #filters: 160 | filter size: 3
Adding spectral pool layer 2 | Input size: 20 | filter size: (12,12) | Freq Dropout Bounds: (1.7999999999999998,7.0)
Adding conv layer 3 | Input size: 12 | Input channels: 160 | #filters: 192 | filter size: 3
Adding spectral pool layer 3 | Input size: 12 | filter size: (7,7) | Freq Dropout Bounds: (0.9428571428571428,4.0)
Adding conv layer 4 | Input size: 7 | Input channels: 192 | #filters: 224 | filter size: 3
Adding spectral pool layer 4 | Input size: 7 | filter size: (4,4) | Freq Dropout Bounds: (0.6428571428571429,3.0)
Adding conv layer 5 | Input size: 4 | Input channels: 224 | #filters: 256 | filter size: 3
Adding spectral pool layer 5 | Input size: 4 | filter size:

__Note:__ in the above output, train acc is not same as validation acc even though they are the same because of dropout applied in train and not in validation

In [ ]:
tf.reset_default_graph()
cnn.calc_test_accuracy(xtest, ytest, 'optimal_hyperparams/{0}'.format(full_model_name))

In [ ]:
# Manual tuning
tf.reset_default_graph()
cnn = CNN_Spectral_Pool(M=4,
                        verbose=True,
                        num_output=100,
                        gamma=0.79)

cnn.calc_test_accuracy(xtest, ytest, 'best_model_cifar_100/test_1513439725.3974495')